In [104]:
get_ipython().magic('reset -sf')

/tmp/ipykernel_296506/3674724974.py:1: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('reset -sf')


In [105]:
from IPython import get_ipython
import scipy.io
import numpy as np
import pandas as pd
import scipy.stats as stats
import pyblp

# 1. Setting

A number of national producers sell substitute products in regional markets. The government intends to bail out a struggling firm and and allow it to merge with one of its healthy competitors. What do you expect the welfare consequences to be?

# 2. Data description

For the empirical exercise, we are giving you data on T = 10 markets. In these markets, 11 different firms sell a total of J = 247 products. All of the products are unique, so none of them are offered in multiple markets. The dataset is simulated, but you can still think of a product as a passenger vehicle with a set of characteristics if you like, although the units do not have an interpretation. The dataset contains the following pieces of data, where products are ordered by market (1-10):
- "prodsMarket": T-vector of the number of products in each market
- "share": J-vector of market shares
- "f": J-vector denoting the firm that sells the product
- "ch": J x 4-matrix of constant and three product characteristics
- "pr": J-vector of prices
- "costShifters": J x 2-matrix of cost shifters

# 3. Basic summary statistics

1. Prepare a table with the following pieces of information for each market: How many firms are active? How many products do they market in total? What fraction of agents bought one of the goods in the sample period?

In [ ]:
mat_data = scipy.io.loadmat("/home/hspassos/mestrado/industrial/demand_data.mat")

prodsMarket = {'market': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
prodsMarket = pd.DataFrame(prodsMarket)
prodsMarket['prodsMarket'] = mat_data['prodsMarket'].flatten()

# Criar tabela com os dados por produto
data = pd.DataFrame({
    'market_ids': np.repeat(np.arange(len(prodsMarket)), prodsMarket['prodsMarket']),
    'firm_ids': mat_data['f'].flatten(),
    'shares': mat_data['share'].flatten(),
    'prices': mat_data['pr'].flatten(),
    'const': mat_data['ch'][:, 0],
    'char1': mat_data['ch'][:, 1],
    'char2': mat_data['ch'][:, 2],
    'char3': mat_data['ch'][:, 3],
    'costsh1': mat_data['costShifters'][:, 0],
    'costsh2': mat_data['costShifters'][:, 1],
})


# Tabela com os dados para cada mercado
prodsMarket['activefirms'] = data.groupby('market_ids')['firm_ids'].nunique()

prodsMarket['firm_1'] = data[data['firm_ids'] == 1].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_1'] = data[data['firm_ids'] == 1].groupby('market_ids')['shares'].sum()

prodsMarket['firm_2'] = data[data['firm_ids'] == 2].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_2'] = data[data['firm_ids'] == 2].groupby('market_ids')['shares'].sum()

prodsMarket['firm_3'] = data[data['firm_ids'] == 3].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_3'] = data[data['firm_ids'] == 3].groupby('market_ids')['shares'].sum()

prodsMarket['firm_4'] = data[data['firm_ids'] == 4].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_4'] = data[data['firm_ids'] == 4].groupby('market_ids')['shares'].sum()

prodsMarket['firm_5'] = data[data['firm_ids'] == 5].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_5'] = data[data['firm_ids'] == 5].groupby('market_ids')['shares'].sum()

prodsMarket['firm_6'] = data[data['firm_ids'] == 6].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_6'] = data[data['firm_ids'] == 6].groupby('market_ids')['shares'].sum()

prodsMarket['firm_7'] = data[data['firm_ids'] == 7].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_7'] = data[data['firm_ids'] == 7].groupby('market_ids')['shares'].sum()

prodsMarket['firm_8'] = data[data['firm_ids'] == 8].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_8'] = data[data['firm_ids'] == 8].groupby('market_ids')['shares'].sum()

prodsMarket['firm_9'] = data[data['firm_ids'] == 9].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_9'] = data[data['firm_ids'] == 9].groupby('market_ids')['shares'].sum()

prodsMarket['firm_10'] = data[data['firm_ids'] == 10].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_10'] = data[data['firm_ids'] == 10].groupby('market_ids')['shares'].sum()

prodsMarket['firm_11'] = data[data['firm_ids'] == 11].groupby('market_ids')['firm_ids'].count()
prodsMarket['share_firm_11'] = data[data['firm_ids'] == 11].groupby('market_ids')['shares'].sum()




2. Prepare a table with summary statistics for market share, characteristics, price, and cost shifters. Please include mean, median, minimum, maximum, and standard deviation. You can inspect these statistics separately for each market, but in what you report, you may pool all markets.

In [ ]:
statistics = pd.DataFrame({
    'mean': data.iloc[:, 2:].mean(),
    'median': data.iloc[:, 2:].median(),
    'minimum': data.iloc[:, 2:].min(),
    'maximum': data.iloc[:, 2:].max(),
    'standard deviation': data.iloc[:, 2:].std()
}).T

statistics

# 4. Pure logit model

1. Suppose agents have the following utility function, where i denotes the agent, and j denotes the product: $$u_{ij}= \underbrace{\delta_j}_{x'_j\beta - \alpha_{pj} + \xi} + \epsilon_{ij}$$ where $\epsilon_{ij}$ is an iid error following a standard Type-I Extreme Value distribution with $F(\epsilon) = e^{-e^{-\epsilon}}$ ("logit" errors). Suppose further that the firms know $\xi$ when setting prices but did not know $\xi$ when setting characteristics.

1.a. What statistical assumptions can you make based on this? Which of your conditions, based on data provided to you, identify the parameter vector of interest, $\theta = (\alpha; \beta)$? In other words, what are valid (and relevant) instruments? Is the model over-identified?

                      shares    prices  const     char1     char2     char3  \
mean                0.024299  3.501501    1.0  1.473991  1.498017  1.472444   
median              0.009474  3.520012    1.0  1.425105  1.451934  1.482724   
minimum             0.000124  1.296677    1.0 -1.166353 -1.147205 -1.387708   
maximum             0.268831  6.212918    1.0  4.362983  3.936271  4.482867   
standard deviation  0.038427  0.828860    0.0  0.967242  0.913044  0.985834   

                     costsh1   costsh2  
mean                0.264297  0.238784  
median              0.272270  0.236695  
minimum             0.005698  0.002097  
maximum             0.499834  0.499676  
standard deviation  0.140623  0.139543  


1.b. Show how you can invert market shares to obtain the mean utility level $\delta_j$ for each product.

In [ ]:
logit_formulation = pyblp.Formulation('prices + char1 + char2 + char3 + costsh1 + costsh2', absorb='C(market_ids)')

problem = pyblp.Problem(logit_formulation, data)

logit_results = problem.solve()

Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N    F    K1    MD    ED 
---  ---  ---  ----  ----  ----
10   247  11    6     5     1  

Formulations:
     Column Indices:          0       1      2      3       4        5   
--------------------------  ------  -----  -----  -----  -------  -------
X1: Linear Characteristics  prices  char1  char2  char3  costsh1  costsh2
Solving the problem ...
Updating the weighting matrix ...
Computed results after 00:00:00.

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix
Step      Value      Shares   Condition Number
----  -------------  -------  ----------------
 1    +2.718427E-27     0      +9.157910E+01  

Estimating standard errors ...
Computed results after 00:00:00.

Problem Results Summary:
GMM     Objective    Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Shares   Condition Number  Condition Number 
----  -----------

The model may be under-identified. The total number of unfixed parameters is 6, which is more than the total number of moments, 5. Consider checking whether instruments were properly specified when initializing the problem, and whether parameters were properly configured when solving the problem.


1.c. Estimate $\theta = (\alpha; \beta)$ and provide standard errors for your estimate. You can try different combinations of instruments, but please use all the different types of instruments that are included or can be constructed from the data (i.e., "BLP instruments").

In [ ]:

data['delta'] = logit_results.compute_delta()
beta_est = logit_results.beta
alpha_est = logit_results.sigma
#standard_errors = logit_results.compute_standard_errors()

#cov_matrix = logit_results.covariance



2. Estimate and present the matrix of cross- and own-price elasticities for market 10 based on your model and parameter estimates.

In [109]:
elasticities = logit_results.compute_elasticities()
elasticities_market_10 = elasticities[data.market_ids == 9]


Computing elasticities with respect to prices ...
Finished after 00:00:00.



3. In the next question, we are going to free up the substitution pattern by introducing random coefficients as in BLP. Alternatively, we could think about implementing nested logit, the pure characteristics model, or multinomial probit. Would they be appealing in this setting? Why or why not?

# 5. Random Coefficient model

1. Suppose agents have the following utility function, where i denotes the agent, and j denotes the product: $$u_{ij} = \underbrace{\delta_j}_{x'_j-\alpha p_j+\xi_j} + \sum_{k \in \{1,2\}}{\sigma_k \nu_{ik} x_{jk} - \sigma_p \nu_{ip} p_{j} + \epsilon_{ij}}$$ where $epsilon_{ij}$ is an iid error following a standard Type-I Extreme Value distribution, and $\nu_{i,\cdot} \overset {iid}\sim  \mathcal{N}(0,1)$ is an iid standard normal error. To summarize: the model is as before, but with random coefficients on the constant, the first characteristic, and price. The orthogonality/exogeneity assumptions
remain the same as before.

1.a. What is the contraction mapping used here for the inner loop? Is there a way to reduce the computational burden from the contraction mapping? (Hint: take a look at page 4 of the appendix to Nevo (2000).) In the following, make sure to set the \inner tolerance" level for the contraction mapping very tight, in your final run ideally on the order of $10^{-14}$.


In [ ]:
product_formulations = (
    pyblp.Formulation('0 + prices', absorb='C(market_ids)'),
    pyblp.Formulation('1 + prices + char1 + char2 + char3 + costsh1 + costsh2')`
)
agent_formulation = pyblp.Formulation('0 + income + income_squared + age + child')
problem = pyblp.Problem(product_formulations, product_data, agent_formulation, agent_data)
problem

In [59]:
elasticities = results.compute_elasticities()
diversions = results.compute_diversion_ratios()

NameError: name 'results' is not defined

In [ ]:
formulation = pyblp.Formulation('0 + prices + char1 + char2 + char3')
local_instruments = pyblp.build_differentiation_instruments(
formulation,
data
)
local_instruments

quadratic_instruments = pyblp.build_differentiation_instruments(
formulation,
data,
version='quadratic'
)
quadratic_instruments

for i in range(8):
    del data[f'demand_instruments{i}']

for i, column in enumerate(local_instruments.T):
    data[f'demand_instruments{i}'] = column
data

,market_ids,firm_ids,shares,prices,char1,char2,char3,char4,costsh1,costsh2,demand_instruments0,demand_instruments1,demand_instruments2,demand_instruments3,demand_instruments4,demand_instruments5,demand_instruments6,demand_instruments7
0,0,1,0.002115,4.775915,1.0,2.750983,0.672526,2.139236,0.158296,0.264869,1.0,0.0,0.0,1.0,10.0,0.0,12.0,19.0
1,0,9,0.032741,3.388449,1.0,1.022277,2.476778,2.685004,0.479222,0.151009,2.0,0.0,1.0,0.0,25.0,0.0,22.0,17.0
2,0,9,0.009585,4.085734,1.0,2.667053,0.760962,1.980984,0.276290,0.002862,2.0,0.0,1.0,1.0,23.0,0.0,14.0,21.0
3,0,7,0.024101,2.432229,1.0,1.068635,0.910503,0.036386,0.109393,0.108614,2.0,0.0,3.0,3.0,17.0,0.0,20.0,19.0
4,0,2,0.012206,4.486977,1.0,2.138262,1.871875,3.655649,0.064100,0.468338,1.0,0.0,1.0,2.0,16.0,0.0,20.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,9,2,0.006462,4.524368,1.0,3.006674,0.118876,1.877790,0.480930,0.039359,0.0,0.0,0.0,0.0,5.0,0.0,3.0,4.0
243,9,10,0.002214,4.118809,1.0,1.125124,1.748192,1.067760,0.483401,0.003548,0.0,0.0,0.0,0.0,7.0,0.0,7.0,9.0
244,9,3,0.103526,2.840987,1.0,2.284540,0.467589,1.036729,0.018995,0.136472,1.0,0.0,1.0,1.0,5.0,0.0,7.0,5.0
245,9,11,0.003856,4.967610,1.0,2.869888,0.569319,2.678046,0.252503,0.184921,1.0,0.0,1.0,0.0,2.0,0.0,3.0,7.0
